# Линейный SVM "своими руками"

##Генерируем обучающую и тестовую выборку для экспериментов

In [107]:
from sklearn.cross_validation import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(n_samples=10000, n_features=20, n_classes=2, n_informative=20, n_redundant=0,
                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

print len(X), len(y)
print len(X_train)

10000 10000
8000


## Пишем свой класс для SVM

In [108]:
import numpy as np
import random
from random import randint

np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return self.w * (1.0 / self.C)

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -1.0 if 1 - answer * self.f(x) > 0 else 0.0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            self.w -= (x * step * y * self.der_loss(x, y) + self.der_reg())
            
            # w_0 update
            self.w0 -= step * y * self.der_loss(x, y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [109]:
model = MySVM()
model.fit(X_train, y_train)
print model.w, model.w0

[ 0.35257911 -0.27479344  0.01739151  0.49542815 -0.03881673  0.60642702
 -0.05514296 -0.11341681  0.05722209 -1.0234016  -0.03732896 -0.12044018
  0.33696173 -0.09902652 -0.60573299 -0.40494819  0.59822001  0.41865654
  0.50763925  0.57967597] -0.943505480738


In [110]:
predictions = model.predict(X_test)

In [111]:
print predictions

[1 0 0 ..., 1 1 0]


In [112]:
print y_test, len(y_test), sum(y_test)

[1 0 1 ..., 1 0 1] 2000 991


In [113]:
print len(predictions), sum(predictions)

2000 991


Вычислим accuracy на нашем датасете:

In [114]:
print sum(predictions == y_test) / float(len(y_test))

0.676
